In [1]:
# ===========================
# INSTALL REQUIRED PACKAGES (Run once)
# ===========================
# !pip install python-telegram-bot==20.4
# !pip install gspread oauth2client
# !pip install nest_asyncio


# ===========================
# IMPORTS
# ===========================
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import (
    ApplicationBuilder, CommandHandler, MessageHandler,
    ConversationHandler, filters
)
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import nest_asyncio
import asyncio


# ===========================
# CONFIG
# ===========================
BOT_TOKEN = "8551540016:AAEGszWLXmeZ7Ioa4FL-HgJ52lAkKehP9NQ"
CREDENTIAL_FILE = "vaanidukaan_cred.json"
SPREADSHEET_NAME = "Impact_1"

BUSINESS_SHEET_NAME = "Business_Registration"
PRODUCT_SHEET_NAME = "Product_Details"
ORDER_SHEET_NAME = "Order_Details"

# NEW → Marketing Contacts Sheet
MARKETING_SPREADSHEET = "Marketing_info"
CONTACT_SHEET_NAME = "User_details"


# ===========================
# GOOGLE SHEET SETUP
# ===========================
scope = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]
creds = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIAL_FILE, scope)
client = gspread.authorize(creds)

business_sheet = client.open(SPREADSHEET_NAME).worksheet(BUSINESS_SHEET_NAME)
product_sheet = client.open(SPREADSHEET_NAME).worksheet(PRODUCT_SHEET_NAME)
order_sheet = client.open(SPREADSHEET_NAME).worksheet(ORDER_SHEET_NAME)
contact_sheet = client.open(MARKETING_SPREADSHEET).worksheet(CONTACT_SHEET_NAME)


# ===========================
# STATES
# ===========================
(
    CHOOSE_CATEGORY,
    CHOOSE_PRODUCT,
    ENTER_QTY,
    ENTER_NAME,
    ENTER_PHONE,
    ENTER_ADDRESS,
    CHOOSE_PAYMENT,
    CONFIRM_UPI_PAYMENT,
    CONFIRM_ORDER
) = range(9)


CATEGORY_MAP = {
    "1": "Papad",
    "2": "Pickle",
    "3": "Ghee",
    "4": "Handicrafts"
}


# ===========================
# RESET to CATEGORY MENU
# ===========================
async def go_to_category(update: Update, context):
    context.user_data.clear()
    return await start(update, context)


# ===========================
# START BOT
# ===========================
async def start(update: Update, context):
    context.user_data.clear()

    try:
        with open("logo_cust_1.jpg", "rb") as img:
            await update.message.reply_photo(
                photo=img,
                caption="👋 Welcome to Vapyar Saathi Marketplace!"
            )
    except:
        await update.message.reply_text("👋 Welcome to Vapyar Saathi Marketplace!")

    await update.message.reply_text(
        "Select a category:\n"
        "1️⃣ Papad\n"
        "2️⃣ Pickle\n"
        "3️⃣ Ghee\n"
        "4️⃣ Handicrafts\n\n"
        "Reply with 1-4:"
    )
    return CHOOSE_CATEGORY


# ===========================
# CHOOSE CATEGORY
# ===========================
async def choose_category(update: Update, context):
    choice = update.message.text.strip()
    if choice not in CATEGORY_MAP:
        await update.message.reply_text("❌ Choose a valid category (1-4)")
        return CHOOSE_CATEGORY

    category = CATEGORY_MAP[choice]
    all_products = product_sheet.get_all_records()

    available = [
        p for p in all_products
        if p.get("Category", "").lower() == category.lower()
        and p.get("Stock", "0") not in ("0", "")
    ]

    if not available:
        await update.message.reply_text(f"❌ No {category} available.")
        return CHOOSE_CATEGORY

    biz_data = business_sheet.get_all_records()
    biz_map = {str(b["Business_ID"]): b for b in biz_data}

    context.user_data["product_list"] = available
    context.user_data["business_map"] = biz_map

    await update.message.reply_text(f"🛒 {category} Products:\n")

    for i, p in enumerate(available, 1):
        b = biz_map.get(str(p["Business_ID"]), {})
        caption = (
            f"{i}️⃣ {p['Product_Name']}\n"
            f"🏪 {b.get('Business_name','N/A')} — {b.get('Location','')}\n"
            f"💰 ₹{p['Price']} ({p['Weight_Size']})\n"
            f"📦 Stock: {p['Stock']}"
        )
        img = p.get("Images", "")
        try:
            if img: await update.message.reply_photo(photo=img, caption=caption)
            else: await update.message.reply_text(caption)
        except:
            await update.message.reply_text(caption)

    await update.message.reply_text("Reply with product number:")
    return CHOOSE_PRODUCT


# ===========================
# CHOOSE PRODUCT
# ===========================
async def choose_product(update, context):
    msg = update.message.text.strip()
    if not msg.isdigit():
        await update.message.reply_text("Enter a valid number")
        return CHOOSE_PRODUCT

    idx = int(msg)
    products = context.user_data["product_list"]

    if idx < 1 or idx > len(products):
        await update.message.reply_text("Invalid choice")
        return CHOOSE_PRODUCT

    context.user_data["selected_product"] = products[idx - 1]
    await update.message.reply_text("Enter quantity:")
    return ENTER_QTY


# ===========================
# ENTER QUANTITY
# ===========================
async def enter_qty(update, context):
    qty = update.message.text.strip()
    if not qty.isdigit():
        await update.message.reply_text("Enter a number")
        return ENTER_QTY

    qty = int(qty)
    p = context.user_data["selected_product"]

    if qty <= 0 or qty > int(p["Stock"]):
        await update.message.reply_text(f"Max allowed: {p['Stock']}")
        return ENTER_QTY

    context.user_data["quantity"] = qty
    await update.message.reply_text("Enter your name:")
    return ENTER_NAME


# ===========================
# ENTER NAME
# ===========================
async def enter_name(update, context):
    context.user_data["customer_name"] = update.message.text.strip()
    await update.message.reply_text("Enter phone:")
    return ENTER_PHONE


# ===========================
# ENTER PHONE
# ===========================
async def enter_phone(update, context):
    context.user_data["customer_phone"] = update.message.text.strip()
    await update.message.reply_text("Enter delivery address:")
    return ENTER_ADDRESS


# ===========================
# ENTER ADDRESS → PAYMENT
# ===========================
async def enter_address(update, context):
    context.user_data["customer_address"] = update.message.text.strip()
    await update.message.reply_text(
        "Select Payment Method:\n"
        "1️⃣ UPI\n"
        "2️⃣ Cash On Delivery"
    )
    return CHOOSE_PAYMENT


# ===========================
# PAYMENT SELECTED
# ===========================
async def choose_payment(update, context):
    choice = update.message.text.strip()
    p = context.user_data["selected_product"]

    if choice == "1":
        biz_id = str(p["Business_ID"])
        all_biz = business_sheet.get_all_records()
        owner = next((b for b in all_biz if str(b["Business_ID"]) == biz_id), None)
        upi = owner.get("UPI_ID", "Not Provided")

        context.user_data["payment_mode"] = "UPI"
        context.user_data["upi_id"] = upi

        await update.message.reply_text(
            f"💳 UPI ID:\n`{upi}`\n\n"
            "After payment reply:\n"
            "1️⃣ Paid\n2️⃣ Cancel",
            parse_mode="Markdown"
        )
        return CONFIRM_UPI_PAYMENT

    elif choice == "2":
        context.user_data["payment_mode"] = "COD"
        return await show_order_summary(update, context)

    await update.message.reply_text("Enter 1 or 2")
    return CHOOSE_PAYMENT



# ===========================
# CONFIRM UPI PAYMENT
# ===========================
async def confirm_upi_payment(update, context):
    choice = update.message.text.strip()
    if choice == "2":
        await update.message.reply_text("❌ Payment cancelled.")
        return ConversationHandler.END

    if choice == "1":
        return await show_order_summary(update, context)

    await update.message.reply_text("Enter 1 or 2")
    return CONFIRM_UPI_PAYMENT



# ===========================
# ORDER SUMMARY  ⭐ FIXED ⭐
# ===========================
async def show_order_summary(update, context):
    p = context.user_data["selected_product"]
    q = context.user_data["quantity"]

    # CLEAN PRICE VALUE → Fixes the error
    price_value = ''.join(filter(str.isdigit, str(p["Price"])))
    total = int(price_value) * q  # Safe conversion now
    context.user_data["amount"] = total

    await update.message.reply_text(
        f"🧾 Order Summary:\n\n"
        f"{p['Product_Name']} x{q}\n"
        f"Total: ₹{total}\n\n"
        "1️⃣ Confirm\n2️⃣ Cancel"
    )

    return CONFIRM_ORDER



# ===========================
# CONFIRM ORDER
# ===========================
async def confirm_order(update, context):
    if update.message.text.strip() == "2":
        await update.message.reply_text("❌ Order Cancelled.")
        return ConversationHandler.END

    p = context.user_data["selected_product"]
    q = context.user_data["quantity"]
    name = context.user_data["customer_name"]
    phone = context.user_data["customer_phone"]
    address = context.user_data["customer_address"]
    amount = context.user_data["amount"]
    business_id = p["Business_ID"]
    product_id = p["Product_ID"]
    payment_mode = context.user_data.get("payment_mode", "Unknown")

    orders = order_sheet.get_all_records()
    count = sum(1 for o in orders if str(o["Business_ID"]) == str(business_id))
    order_id = f"{business_id}_oid{count + 1}"

    order_sheet.append_row([
        order_id, name, phone, p["Product_Name"], product_id,
        q, amount, "Pending", "Pending", address, business_id, payment_mode
    ])

    # Update Stock
    ids = product_sheet.col_values(1)
    if product_id in ids:
        row = ids.index(product_id) + 1
        product_sheet.update_cell(row, 6, int(p["Stock"]) - q)

    # ⭐ Add user to Marketing sheet
    chat_id = update.effective_chat.id
    contacts = contact_sheet.get_all_records()
    phones = [str(c.get("Phone_Number", "")) for c in contacts]

    if phone not in phones:
        contact_sheet.append_row([name, phone, chat_id])
    else:
        idx = phones.index(phone) + 2
        contact_sheet.update_cell(idx, 3, chat_id)

    keyboard = [
        [InlineKeyboardButton("🎁 Get More Offers!", url="https://t.me/Vyapar_Sathi_Marketing_bot?start=joined")]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)

    await update.message.reply_text(
        f"🎉 Order Confirmed!\n🆔 Order ID: {order_id}\n\n"
        "Tap below to get exclusive offers 👇",
        reply_markup=reply_markup
    )
    return ConversationHandler.END


# ===========================
# CANCEL
# ===========================
async def cancel(update, context):
    await update.message.reply_text("❌ Cancelled.")
    return ConversationHandler.END


# ===========================
# BOT RUN
# ===========================
app = ApplicationBuilder().token(BOT_TOKEN).build()

app.add_handler(MessageHandler(filters.Regex("(?i)^(category|menu)$"), go_to_category))

app.add_handler(ConversationHandler(
    entry_points=[CommandHandler("start", start)],
    states={
        CHOOSE_CATEGORY: [MessageHandler(filters.TEXT, choose_category)],
        CHOOSE_PRODUCT: [MessageHandler(filters.TEXT, choose_product)],
        ENTER_QTY: [MessageHandler(filters.TEXT, enter_qty)],
        ENTER_NAME: [MessageHandler(filters.TEXT, enter_name)],
        ENTER_PHONE: [MessageHandler(filters.TEXT, enter_phone)],
        ENTER_ADDRESS: [MessageHandler(filters.TEXT, enter_address)],
        CHOOSE_PAYMENT: [MessageHandler(filters.TEXT, choose_payment)],
        CONFIRM_UPI_PAYMENT: [MessageHandler(filters.TEXT, confirm_upi_payment)],
        CONFIRM_ORDER: [MessageHandler(filters.TEXT, confirm_order)],
    },
    fallbacks=[CommandHandler("cancel", cancel)],
    allow_reentry=True
))

nest_asyncio.apply()
print("🚀 Customer Bot Started… Running Polling now!")
asyncio.get_event_loop().run_until_complete(app.run_polling())


🚀 Customer Bot Started… Running Polling now!


RuntimeError: Cannot close a running event loop

🚀 Customer Bot Started… Running Polling now!


RuntimeError: Cannot close a running event loop